# 4a5e1

This is a reference to the Bitcoin's genesis block hash. These are the first five hex-encoded characters (depending on the endianness you're using). That is our first clue.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import dbm

with open("posts.json", "r") as f:
    post_data = json.load(f)

titles = []
bodies = []
footers = []
for post in post_data:
    titles.append(post["title"])
    bodies.append(post["body"])
    footers.append(post["footer"])


## Decoding the Bodies

In [ ]:
def apiGetBlockHash(height):
    url = f"https://mempool.space/api/block-height/{height}"
    resp = requests.get(url=url)

    if resp.status_code != 200:
        print(f"error getting data: status_code {resp.status_code}")
        raise KeyboardInterrupt
    else:
        blockhash = resp.content.decode()

    return blockhash


def apiGetTxId(blockhash, txindex):
    url = f"https://mempool.space/api/block/{blockhash}/txid/{txindex}"
    resp = requests.get(url=url)

    if resp.status_code != 200:
        print(f"error getting data: status_code {resp.status_code}")
        raise KeyboardInterrupt
    else:
        txid = resp.content.decode()

    return txid

### Layer 1

By breaking up the body of each post into groups of three (block height, transaction number, index of hex character), we get the following hex-encoded strings:

In [ ]:
with dbm.open('kvs_height_blockhash', 'c') as kvs_height_blockhash:
    with dbm.open('kvs_blockhash_txindex_txid', 'c') as kvs_blockhash_txindex_txid:
        for n, body in enumerate(bodies):

            x = [int(x) for x in body.split("-")]
            g = [(x[i], x[i+1], x[i+2]) for i in range(0, len(x), 3)]
            
            hexstr = ""
            for height, txindex, i in g:
                
                if str(height) in kvs_height_blockhash:
                    blockhash = kvs_height_blockhash[str(height)].decode()
                else:
                    blockhash = apiGetBlockHash(height)
                    kvs_height_blockhash[str(height)] = blockhash

                if f"{blockhash}_{txindex}" in kvs_blockhash_txindex_txid:
                    txid = kvs_blockhash_txindex_txid[f"{blockhash}_{txindex}"].decode()
                else:
                    txid = apiGetTxId(blockhash, txindex)
                    kvs_blockhash_txindex_txid[f"{blockhash}_{txindex}"] = txid

                hexstr += txid[i]

            print(f"{n+1}. {bytes.fromhex(hexstr).decode()}")

Note that most entries are the same, but 6 is different. It provides the "magic number" 5487-3. (I called it that because that's what it was called in Layer 2 below.)

### Layer 2

By subtracting pairs of tuples, we get a new set of tuples that provides more hex encoded strings:

In [ ]:
with dbm.open('kvs_height_blockhash', 'c') as kvs_height_blockhash:
    with dbm.open('kvs_blockhash_txindex_txid', 'c') as kvs_blockhash_txindex_txid:
        for n, body in enumerate(bodies):

            x = [int(x) for x in body.split("-")]
            g = [(x[i], x[i+1], x[i+2]) for i in range(0, len(x), 3)]
            
            # here is the "basic math referenced"
            gg = [(g[x+1][0] - g[x][0],g[x+1][1] - g[x][1],g[x+1][2] - g[x][2],) for x in range(0, len(g), 2)]

            hexstr = ""
            for height, txindex, i in gg:
                
                if str(height) in kvs_height_blockhash:
                    blockhash = kvs_height_blockhash[str(height)].decode()
                else:
                    blockhash = apiGetBlockHash(height)
                    kvs_height_blockhash[str(height)] = blockhash

                if f"{blockhash}_{txindex}" in kvs_blockhash_txindex_txid:
                    txid = kvs_blockhash_txindex_txid[f"{blockhash}_{txindex}"].decode()
                else:
                    txid = apiGetTxId(blockhash, txindex)
                    kvs_blockhash_txindex_txid[f"{blockhash}_{txindex}"] = txid
                    
                hexstr += txid[i]
            
            print(f"{n+1}. {bytes.fromhex(hexstr).decode()}")



These quotes are attributed to the following people:
- Albert Einstein [1-4]
- Ursula K. Le Guin [13]
- Confucious [14]
- Walter Elliot [15]
- Roy T. Bennett [16]

Einstein's birthday is March 14, 1879. This is 3/14, presumably meaning we will need to look at value of $\pi$.

What do these "magic numbers" mean?

## New Territory

The code below is speculative and hasn't led to further clues yet.

## Magic Numbers

Based on my conversation with the person who made this puzzle, I know the titles have something to do with the decimal digits of pi. Beyond that, I'm not sure. I don't know if the magic numbers are involved in this process or not.

In [ ]:
# load the decimimal representation of pi
with open("pi.txt", "r") as f:
    pistr = f.read()
    # pistr = pistr[0] + pistr[2:] # remove decimal point

magic_numbers = [
    (2048, 3),  # from Layer 2
    (780000, 4),  # from Layer 2
    (123, 4),  # from Layer 2
    (8976, 4),  # from Layer 2
    (0, 0),  # ???
    (5487, 3),  # from Layer 1
    (1678, 4),  # from Layer 2
    (123456, 3),  # from Layer 2
    (11, 4),  # from Layer 2
    (0, 0),  # ???
    (123, 5),  # from Layer 2
    (1891, 3),  # from Layer 2
    (902101, 4),  # from Layer 2
    (761761, 3),  # from Layer 2
    (12121, 2),  # from Layer 2
    (12312, 3),  # from Layer 2
]

for mn in magic_numbers:
    print(pistr[mn[0]:mn[0]+mn[1]])

## Decoding the Titles

My attempts at doing so. The code below hasn't produced anything meaningful yet.

### Distribution of Numbers in Titles

In [ ]:
occurances = dict()
for title in titles:
    ds = [int(s) for s in title.split("-")]
    for d in ds:
        if d in occurances.keys():
            occurances[d] += 1
        else:
            occurances[d] = 1

x = np.array([x[0] for x in sorted(occurances.items())])
y = np.array([x[1] for x in sorted(occurances.items())])
y = y / np.sum(y)

z = np.cumsum(y)

plt.figure()

plt.subplot(2,1,1)
plt.plot(x, y / sum(y)) # probability mass function

plt.subplot(2,1,2)
plt.plot(x, z) # cumulative mass function
plt.hlines([0.5], 0, np.max(x))

plt.show()

Based on this distribution, I suspect that these numbers indicate how many digits ahead to look from the current digit of pi. The trick is knowing where to start looking.

In [ ]:
occurances = dict()
for i in range(10):
    occurances[i] = 0

i = 0
for title in sorted(titles, key=lambda title: int(title.split("-")[0])):
    ds = [int(s) for s in title.split("-")]
    # print(ds[0], end=": ")
    
    print(ds[0], end=": ")
    for d in ds:
        i += d
        digit = int(pistr[i])
        print(digit, end="")
        occurances[digit] += 1
    print()

x = [x[0] for x in sorted(occurances.items())]
y = [x[1] for x in sorted(occurances.items())]

plt.figure()
plt.plot(x,y)
plt.ylim([0, 100])
plt.show()